**This is the task for the second home work (HW2)**

Need to get Armenian MCV dataset and train Armenian ASR model

Quality metric is WER on Armenian MCV test subset.

In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2b7a8934939a0821d290d412d66598d888f606869e0102bea4ef86212a87521c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3
  libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox-fmt-mp3 libsox3 libwavpack1 sox
0 upgraded, 10 newly installed, 0 to remove and 45 not upgraded.
Need to get 72

'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [2]:
import os
import glob
import subprocess
import tarfile
import wget
import copy
from omegaconf import OmegaConf, open_dict


In [3]:
data_dir = 'datasets/'

if not os.path.exists(data_dir):
  os.makedirs(data_dir, exist_ok=True)

if not os.path.exists("scripts"):
  os.makedirs("scripts")

import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager

[NeMo W 2024-04-20 08:11:17 nemo_logging:349] /usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
      self.pid = os.fork()
    


**Download dataset**

We will use the NeMo script in the scripts directory to download and prepare the Mozilla Common Voice (MCV) dataset for Armenian.

The data preparation script will download the audio files and respective transcripts and then process the audio into mono-channel 16 kHz wave files that can be easily used for training ASR models.

**Hugging Face**

Now, let's download the Mozilla CommonVoice Spanish dataset. We will ignore the larger train file and get just the test part for the purposes of this tutorial. For good results, you will need to get the train files and likely other datasets too, bringing the total to over 1k hours.

Website steps:

Visit https://huggingface.co/settings/profile

Visit "Access Tokens" on list of items.

Create new token - provide a name for the token and "read" access is sufficient.

PRESERVE THAT TOKEN API KEY. You can copy that key for next step.

Visit the HuggingFace Dataset page for [Mozilla Common Voice 16.1](https://huggingface.co/datasets/mozilla-foundation/common_voice_16_1)

There should be a section that asks you for your approval.

Make sure you are logged in and then read that agreement.

If and only if you agree to the text, then accept the terms.

Code steps:

* Now below, run login()

* Paste your preserved HF TOKEN API KEY to the text box."

In [4]:
from huggingface_hub import login
login()

In [14]:
VERSION = "mozilla-foundation/common_voice_16_1"
LANGUAGE = "hy-AM"

In [15]:
tokenizer_dir = os.path.join('tokenizers', LANGUAGE)
manifest_dir = os.path.join('datasets', LANGUAGE, VERSION, LANGUAGE)

In [16]:
# If something goes wrong during data processing, un-comment the following line to delete the cached dataset
# !rm -rf datasets/$LANGUAGE
!mkdir -p datasets

The following cell will download the Japanese MCV corpus, preprocess the audio and prepare manifest files that can be directly used by NeMo models.

We will use the convert_hf_dataset_to_nemo.py script located in the nemo/scripts/speech_recognition dir if you cloned NeMo repo

In [17]:
if not os.path.exists("convert_hf_dataset_to_nemo.py"):
    !wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/speech_recognition/convert_hf_dataset_to_nemo.py

In [18]:
!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="train" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="validation" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="test" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="other" \
    ensure_ascii=False \
    use_auth_token=True

!python convert_hf_dataset_to_nemo.py \
    output_dir=datasets/$LANGUAGE \
    path=$VERSION \
    name=$LANGUAGE \
    split="invalidated" \
    ensure_ascii=False \
    use_auth_token=True

/content/convert_hf_dataset_to_nemo.py:346: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_name='hfds_config', config_path=None)
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load 

In [19]:
train_manifest = f"{manifest_dir}/train/train_mozilla-foundation_common_voice_16_1_manifest.json"
dev_manifest = f"{manifest_dir}/validation/validation_mozilla-foundation_common_voice_16_1_manifest.json"
test_manifest = f"{manifest_dir}/test/test_mozilla-foundation_common_voice_16_1_manifest.json"
other_manifest = f"{manifest_dir}/other/other_mozilla-foundation_common_voice_16_1_manifest.json"
invalidated_manifest = f"{manifest_dir}/invalidated/invalidated_mozilla-foundation_common_voice_16_1_manifest.json"

In [20]:
train_manifest_full = f"{manifest_dir}/train_full_mozilla-foundation_common_voice_16_1_manifest.json"
!cat $train_manifest $other_manifest $invalidated_manifest > $train_manifest_full

In [21]:
import json
import string
def preprocess_data(manifest, output_manifet_filename, encoding='utf-8'):
  updated_manifest = []
  punctuation = string.punctuation + '։՛՞-'
  with open(manifest, "r") as manifest_file:
    for sample in manifest_file:
      data = json.loads(sample)
      text = data["text"]
      text = text.lower()
      text = text.translate(str.maketrans('', '', punctuation))
      data["text"] = text
      updated_manifest.append(data)

  with open(output_manifet_filename, "w", encoding='utf-8') as manifest_file:
    for sample in updated_manifest:
      json.dump(sample, manifest_file)
      manifest_file.write("\n")

In [22]:
preprocess_data(train_manifest_full, f"{manifest_dir}/train/train_manifest.json")
preprocess_data(dev_manifest, f"{manifest_dir}/validation/dev_manifest.json")
preprocess_data(test_manifest, f"{manifest_dir}/test/test_manifest.json")

train_manifest_full = f"{manifest_dir}/train/train_manifest.json"
dev_manifest = f"{manifest_dir}/validation/dev_manifest.json"
test_manifest = f"{manifest_dir}/test/test_manifest.json"

**Hint**: Convert texts to lowercase and remove punctuation to improve WER.

In [23]:
if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tokenizers/process_asr_text_tokenizer.py

--2024-04-20 08:41:32--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16631 (16K) [text/plain]
Saving to: ‘scripts/process_asr_text_tokenizer.py’

process_asr_text_to 100%[===================>]  16.24K  --.-KB/s    in 0.001s  

2024-04-20 08:41:32 (22.8 MB/s) - ‘scripts/process_asr_text_tokenizer.py’ saved [16631/16631]



**Hint**: Play with `VOCAB_SIZE` to improve WER.

In [24]:
TOKENIZER_TYPE = "bpe" # "bpe", "unigram"
VOCAB_SIZE = 128 + 2

In [25]:
!python scripts/process_asr_text_tokenizer.py \
  --manifest=$train_manifest_full,$dev_manifest \
  --vocab_size=$VOCAB_SIZE \
  --data_root=$tokenizer_dir \
  --tokenizer="spe" \
  --spe_type=$TOKENIZER_TYPE \
  --spe_character_coverage=1.0 \
  --no_lower_case \
  --log

[NeMo W 2024-04-20 08:41:39 nemo_logging:349] /usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
      self.pid = os.fork()
    
INFO:root:Finished extracting manifest : datasets/hy-AM/mozilla-foundation/common_voice_16_1/hy-AM/train/train_manifest.json
INFO:root:Finished extracting manifest : datasets/hy-AM/mozilla-foundation/common_voice_16_1/hy-AM/validation/dev_manifest.json
INFO:root:Finished extracting all manifests ! Number of sentences : 12707
[NeMo I 2024-04-20 08:41:40 sentencepiece_tokenizer:317] Processing tokenizers/hy-AM/text_corpus/document.txt and store at tokenizers/hy-AM/tokenizer_spe_bpe_v130
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokenizers/hy-AM/text_corpus/document.txt --model_prefix=tokenizers/hy-AM/tokenizer_spe_bpe_v130/tokenizer --vocab_size=130 --shuffle_input_sentence=true --ha

**Hint**: Try different models.

In [26]:
model = nemo_asr.models.ASRModel.from_pretrained("stt_en_fastconformer_ctc_large", map_location='cpu')

[NeMo I 2024-04-20 08:41:43 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_fastconformer_ctc_large/versions/1.0.0/files/stt_en_fastconformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_fastconformer_ctc_large/00a071a9dac048acc3aeea942b0bfa40/stt_en_fastconformer_ctc_large.nemo
[NeMo I 2024-04-20 08:41:45 common:815] Instantiating model from pre-trained checkpoint
[NeMo I 2024-04-20 08:41:47 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-04-20 08:41:48 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-04-20 08:41:48 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2024-04-20 08:41:48 features:289] PADDING: 0
[NeMo I 2024-04-20 08:41:51 save_restore_connector:263] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_fastconformer_ctc_large/00a071a9dac048acc3aeea942b0bfa40/stt_en_fastconformer_ctc_large.nemo.


In [27]:
import torch
import torch.nn as nn

freeze_encoder = True # set to False if dare lol

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

if freeze_encoder:
  model.encoder.freeze()
  model.encoder.apply(enable_bn_se)
  logging.info("Model encoder has been frozen")
else:
  model.encoder.unfreeze()
  logging.info("Model encoder has been un-frozen")

[NeMo I 2024-04-20 08:41:51 <ipython-input-27-180610cd99a3>:20] Model encoder has been frozen


In [28]:
TOKENIZER_DIR = os.path.join(tokenizer_dir, f"tokenizer_spe_{TOKENIZER_TYPE}_v{VOCAB_SIZE}")

model.change_vocabulary(new_tokenizer_dir=TOKENIZER_DIR, new_tokenizer_type=TOKENIZER_TYPE)

[NeMo W 2024-04-20 08:41:51 modelPT:258] You tried to register an artifact under config key=tokenizer.model_path but an artifact for it has already been registered.
[NeMo W 2024-04-20 08:41:51 modelPT:258] You tried to register an artifact under config key=tokenizer.vocab_path but an artifact for it has already been registered.
[NeMo W 2024-04-20 08:41:51 modelPT:258] You tried to register an artifact under config key=tokenizer.spe_tokenizer_vocab but an artifact for it has already been registered.


[NeMo I 2024-04-20 08:41:51 mixins:172] Tokenizer SentencePieceTokenizer initialized with 130 tokens
[NeMo I 2024-04-20 08:41:51 ctc_bpe_models:265] 
    Replacing old number of classes (1024) with new number of classes - 130
[NeMo I 2024-04-20 08:41:51 ctc_bpe_models:307] Changed tokenizer to ['<unk>', 'ու', 'ան', 'եր', 'ար', 'ակ', 'ում', '▁է', '▁հ', 'ներ', 'այ', 'ին', '▁ե', 'որ', '▁մ', '▁կ', 'ել', 'ութ', 'ուն', '▁ա', 'ությ', 'ատ', '▁ն', 'ամ', 'ական', '▁բ', '▁տ', '▁գ', 'ած', '▁ս', 'աց', 'աս', 'ով', '▁պ', 'ավ', 'են', '▁դ', 'ություն', '▁են', '▁եւ', 'եւ', 'ագ', 'ների', 'ող', '▁վ', 'իր', '▁այ', 'եղ', 'ես', '▁եր', 'ետ', '▁ան', '▁շ', 'ված', 'եց', 'առ', 'ալ', 'ները', 'ահ', 'աղ', 'ային', 'ման', 'ից', 'ության', '▁նա', '▁մի', '▁լ', '▁թ', 'վում', '▁ար', '▁ք', 'ազ', 'ոն', 'վել', '▁խ', 'րա', '▁որ', 'իս', '▁չ', '▁ու', 'ույ', '▁համ', 'ա', '▁', 'ն', 'ր', 'ո', 'ե', 'ի', 'ւ', 'մ', 'կ', 'տ', 'յ', 'վ', 'ս', 'լ', 'հ', 'ց', 'ը', 'թ', 'է', 'գ', 'դ', 'ղ', 'ք', 'բ', 'պ', 'ծ', 'շ', 'ռ', 'զ', 'խ', 'չ', 'ջ', 'ձ'

In [29]:
cfg = copy.deepcopy(model.cfg)

# Setup new tokenizer
cfg.tokenizer.dir = TOKENIZER_DIR
cfg.tokenizer.type = "bpe"

# Set tokenizer config
model.cfg.tokenizer = cfg.tokenizer

In [30]:
# Setup train/val/test configs
print(OmegaConf.to_yaml(cfg.train_ds))

manifest_filepath: null
sample_rate: 16000
batch_size: 1
shuffle: true
num_workers: 8
pin_memory: true
use_start_end_token: false
trim_silence: false
max_duration: 20
min_duration: 0.1
is_tarred: false
tarred_audio_filepaths: null
shuffle_n: 2048
bucketing_strategy: fully_randomized
bucketing_batch_size: null



In [31]:
# Setup train, validation, test configs
with open_dict(cfg):
  # Train dataset
  cfg.train_ds.manifest_filepath = f"{train_manifest_full},{dev_manifest}"
  cfg.train_ds.batch_size = 32
  cfg.train_ds.num_workers = 8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.use_start_end_token = False
  cfg.train_ds.trim_silence = True

  # Validation dataset
  cfg.validation_ds.manifest_filepath = test_manifest
  cfg.validation_ds.batch_size = 8
  cfg.validation_ds.num_workers = 8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.use_start_end_token = False
  cfg.validation_ds.trim_silence = True

  # Test dataset
  cfg.test_ds.manifest_filepath = test_manifest
  cfg.test_ds.batch_size = 8
  cfg.test_ds.num_workers = 8
  cfg.test_ds.pin_memory = True
  cfg.test_ds.use_start_end_token = False
  cfg.test_ds.trim_silence = True

In [32]:
# setup model with new configs
model.setup_training_data(cfg.train_ds)
model.setup_multiple_validation_data(cfg.validation_ds)
model.setup_multiple_test_data(cfg.test_ds)

[NeMo I 2024-04-20 08:41:52 collections:196] Dataset loaded with 12702 files totalling 18.82 hours
[NeMo I 2024-04-20 08:41:52 collections:197] 5 files were filtered totalling 0.13 hours


[NeMo W 2024-04-20 08:41:52 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


[NeMo I 2024-04-20 08:41:52 collections:196] Dataset loaded with 2853 files totalling 4.55 hours
[NeMo I 2024-04-20 08:41:52 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-04-20 08:41:52 collections:196] Dataset loaded with 2853 files totalling 4.55 hours
[NeMo I 2024-04-20 08:41:52 collections:197] 0 files were filtered totalling 0.00 hours


In [33]:
print(OmegaConf.to_yaml(cfg.optim))

name: adamw
lr: 0.001
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: 15000
  warmup_ratio: null
  min_lr: 0.0001



In [34]:
with open_dict(model.cfg.optim):
  model.cfg.optim.lr = 0.025
  model.cfg.optim.weight_decay = 0.001
  model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  model.cfg.optim.sched.warmup_ratio = 0.10  # 10 % warmup
  model.cfg.optim.sched.min_lr = 1e-9

with open_dict(model.cfg.spec_augment):
  model.cfg.spec_augment.freq_masks = 2
  model.cfg.spec_augment.freq_width = 25
  model.cfg.spec_augment.time_masks = 10
  model.cfg.spec_augment.time_width = 0.05

model.spec_augmentation = model.from_config_dict(model.cfg.spec_augment)

In [35]:
use_cer = False
log_prediction = True

model.wer.use_cer = use_cer
model.wer.log_prediction = log_prediction

In [36]:
import torch
import pytorch_lightning as ptl

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

EPOCHS = 20  # will take approximately 4 hours

trainer = ptl.Trainer(devices=1,
                      accelerator=accelerator,
                      max_epochs=EPOCHS,
                      accumulate_grad_batches=1,
                      enable_checkpointing=False,
                      logger=False,
                      log_every_n_steps=5,
                      check_val_every_n_epoch=10)

# Setup model with the trainer
model.set_trainer(trainer)

# finally, update the model's internal config
model.cfg = model._cfg

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [37]:
from nemo.utils import exp_manager

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/lang-{LANGUAGE}/',
    name=f"ASR-Model-Language-{LANGUAGE}",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

config = OmegaConf.structured(config)

logdir = exp_manager.exp_manager(trainer, config)

[NeMo I 2024-04-20 08:41:53 exp_manager:396] Experiments will be logged at experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-04-20_08-41-53
[NeMo I 2024-04-20 08:41:53 exp_manager:856] TensorboardLogger has been set up


In [38]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir /content/experiments/lang-$LANGUAGE/ASR-Model-Language-$LANGUAGE/
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

<IPython.core.display.Javascript object>

In [39]:
%%time
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-04-20 08:42:05 modelPT:724] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.025
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2024-04-20 08:42:05 lr_scheduler:923] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7b9796c6cca0>" 
    will be used during training (effective maximum steps = 7940) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    min_lr: 1.0e-09
    max_steps: 7940
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 115 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 67.2 K
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
85.6 K    Trainable params
115 M     Non-trainable params
115 M     Total params
460.567   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[NeMo W 2024-04-20 08:42:05 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    
[NeMo W 2024-04-20 08:42:05 nemo_logging:349] /usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
      self.pid = os.fork()
    


...

[NeMo I 2024-04-20 10:16:04 wer:334] 
    
[NeMo I 2024-04-20 10:16:04 wer:335] reference:միջպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամուսնությունները
[NeMo I 2024-04-20 10:16:04 wer:336] predicted:միչպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամունությունները
[NeMo I 2024-04-20 10:16:05 wer:334] 
    
[NeMo I 2024-04-20 10:16:05 wer:335] reference:այդ օրը այլեւս ոչ մի թռչուն չնկատվեց
[NeMo I 2024-04-20 10:16:05 wer:336] predicted:այդտ օը այլեւս ոչ մի թռչչ չունկավեց
[NeMo I 2024-04-20 10:16:05 wer:334] 
    
[NeMo I 2024-04-20 10:16:05 wer:335] reference:երբե՜ք այդ մարդը դաժան չէ այլ...
[NeMo I 2024-04-20 10:16:05 wer:336] predicted:երբեք այդ մարդը դաժան չէ այլ
[NeMo I 2024-04-20 10:16:06 wer:334] 
    
[NeMo I 2024-04-20 10:16:06 wer:335] reference:ագրեգատային վիճակը պայմանավորված է մոլեկուլային զանգվածով
[NeMo I 2024-04-20 10:16:06 wer:336] predicted:աակրեատային վիճակը պայմման աավորված է մոլեկուլային զագգվածով
[NeMo I 2024-04-20 10:16:06 wer:3

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 7940: 'val_wer' reached 0.57283 (best 0.57283), saving model to '/content/experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-04-20_08-41-53/checkpoints/ASR-Model-Language-hy-AM--val_wer=0.5728-epoch=19.ckpt' as top 3


[NeMo I 2024-04-20 10:17:19 nemo_model_checkpoint:217] New best .nemo model saved to: /content/experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-04-20_08-41-53/checkpoints/ASR-Model-Language-hy-AM.nemo


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-04-20_08-41-53/checkpoints/ASR-Model-Language-hy-AM--val_wer=0.5728-epoch=19.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/experiments/lang-hy-AM/ASR-Model-Language-hy-AM/2024-04-20_08-41-53/checkpoints/ASR-Model-Language-hy-AM--val_wer=0.5728-epoch=19.ckpt


CPU times: user 1h 17min 52s, sys: 2min 15s, total: 1h 20min 7s
Wall time: 1h 36min 4s


Please save and download your model.

In [40]:
save_path = f"Model-{LANGUAGE}.nemo"
model.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")

Model saved at path : /content/Model-hy-AM.nemo


In [41]:
trainer.test(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-04-20 10:18:25 wer:334] 
    
[NeMo I 2024-04-20 10:18:25 wer:335] reference:միջպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամուսնությունները
[NeMo I 2024-04-20 10:18:25 wer:336] predicted:միչպետական հարաբերություններում կարեւոր դեր էին խաղում քաղաքական ամունությունները
[NeMo I 2024-04-20 10:18:25 wer:334] 
    
[NeMo I 2024-04-20 10:18:25 wer:335] reference:այդ օրը այլեւս ոչ մի թռչուն չնկատվեց
[NeMo I 2024-04-20 10:18:25 wer:336] predicted:այդտ օը այլեւս ոչ մի թռչչ չունկավեց
[NeMo I 2024-04-20 10:18:25 wer:334] 
    
[NeMo I 2024-04-20 10:18:25 wer:335] reference:երբե՜ք այդ մարդը դաժան չէ այլ...
[NeMo I 2024-04-20 10:18:25 wer:336] predicted:երբեք այդ մարդը դաժան չէ այլ
[NeMo I 2024-04-20 10:18:25 wer:334] 
    
[NeMo I 2024-04-20 10:18:25 wer:335] reference:ագրեգատային վիճակը պայմանավորված է մոլեկուլային զանգվածով
[NeMo I 2024-04-20 10:18:25 wer:336] predicted:աակրեատային վիճակը պայմման աավորված է մոլեկուլային զագգվածով
[NeMo I 2024-04-20 10:18:25 wer:3

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        global_step        │          7940.0           │
│         test_loss         │    0.7553428411483765     │
│         test_wer          │     0.572834849357605     │
└───────────────────────────┴───────────────────────────┘

[{'global_step': 7940.0,
  'test_loss': 0.7553428411483765,
  'test_wer': 0.572834849357605}]